In [1]:
#BATHYMETRY
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
# Configure Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration
# Function to extract coordinates from KML file
def extract_coordinates_from_kml(kml_file):
    coordinates = []
    with open(kml_file, 'r') as file:
        kml_content = file.read()
        # Use regular expression to extract coordinates from KML content
        pattern = r'<coordinates>(.*?)</coordinates>'
        matches = re.findall(pattern, kml_content, re.DOTALL)
        for match in matches:
            # Split coordinates into individual points
            points = match.strip().split()
            for point in points:
                # Split latitude, longitude, and altitude values
                lat, lon, alt = point.split(',')
                coordinates.append((float(lon), float(lat)))
    return coordinates
# Path to the KML file containing coordinates
kml_file_path = 'random.kml'
# Extract coordinates from KML file
coordinates = extract_coordinates_from_kml(kml_file_path)
# Create a WebDriver instance with headless mode
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(20)  # Implicit wait to wait for elements
wait = WebDriverWait(driver, 20)  # Initialize WebDriverWait
driver.get('https://www.advancedconverter.com/map-tools/find-altitude-by-coordinates')
# Open a CSV file for writing
with open('random.csv', 'w', newline='') as csvfile:
    fieldnames = ['Latitude', 'Longitude', 'Depth']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate over coordinates extracted from KML file
    for lat, lon in coordinates:
        # Execute JavaScript to set latitude and longitude values directly
        driver.execute_script(f'document.getElementById("lat").value = "{lat}";')
        driver.execute_script(f'document.getElementById("lng").value = "{lon}";')

        # Click the insert point button
        button = wait.until(EC.element_to_be_clickable((By.ID, 'insertPoint')))
        button.click()

        try:
            # Wait for the depth information to be visible
            elevation = wait.until(EC.visibility_of_element_located((By.ID, 'outputDiv')))
            elevation_text = elevation.find_element(By.TAG_NAME, 'strong').text

            # Extract the depth value from the text
            elevation_value = elevation_text.split()[0]

            # Write to CSV
            writer.writerow({'Latitude': lat, 'Longitude': lon, 'Depth': elevation_value})
        except TimeoutException:
            # If depth information is not available, skip this coordinate
            print(f"Depth data not available for lat: {lat}, lon: {lon}")
driver.quit()
print("Depth data has been saved to csv")

Depth data has been saved to csv


In [9]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from scipy.interpolate import griddata

# Load data
df0 = pd.read_csv("random.csv")
lon = df0["Longitude"]
lat = df0["Latitude"]
elev = df0["Depth"]

# Create grid for surface plot
grid_lon = np.linspace(lon.min(), lon.max(), 100)
grid_lat = np.linspace(lat.min(), lat.max(), 100)
grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
grid_elev = griddata((lon, lat), elev, (grid_lon, grid_lat), method='cubic')

# Create contour plot using Plotly
contour_fig = go.Figure(go.Contour(x=lon, y=lat, z=elev, colorscale='spectral'))
contour_fig.update_layout(title='Bathymetric Contour Map (Plotly)', xaxis_title='Longitude', yaxis_title='Latitude')
contour_fig.update_traces(colorbar=dict(title='Depth', x=1.15))

# Show contour plot
contour_fig.show()

# Constants for converting meters to degrees
# Assuming the area is near the equator for simplicity, adjust as necessary for your region
meters_per_degree_lat = 111320  # Approximate number of meters per degree of latitude
meters_per_degree_lon = 111320  # This varies based on latitude, use the same value for simplicity

# Define the size of the area in meters
area_size_m = 3000
total_area = 3000*3000
# Convert area size to degrees
area_size_deg_lat = area_size_m / meters_per_degree_lat
area_size_deg_lon = area_size_m / meters_per_degree_lon

# Prompt user for the center of the area
center_lon = lon.mean()
center_lat = lat.mean()

# Calculate the boundaries of the area
lon_min = center_lon - area_size_deg_lon / 2
lon_max = center_lon + area_size_deg_lon / 2
lat_min = center_lat - area_size_deg_lat / 2
lat_max = center_lat + area_size_deg_lat / 2

# Filter the dataframe for the selected area
filtered_df = df0[(df0["Longitude"] >= lon_min) & (df0["Longitude"] <= lon_max) & 
                 (df0["Latitude"] >= lat_min) & (df0["Latitude"] <= lat_max)]

# Save the filtered dataframe to a new CSV file
filtered_df.to_csv("filtered_area.csv", index=False)

# Highlight the selected area on the original contour plot
highlight = go.Scatter(
    x=[lon_min, lon_max, lon_max, lon_min, lon_min],
    y=[lat_min, lat_min, lat_max, lat_max, lat_min],
    mode='lines',
    line=dict(color='black', width=2),
    name='Selected Area'
)

contour_fig.add_trace(highlight)
contour_fig.show()

# Create 3D surface plot using Plotly for the selected area
selected_grid_lon = np.linspace(lon_min, lon_max, 100)
selected_grid_lat = np.linspace(lat_min, lat_max, 100)
selected_grid_lon, selected_grid_lat = np.meshgrid(selected_grid_lon, selected_grid_lat)
selected_grid_elev = griddata((lon, lat), elev, (selected_grid_lon, selected_grid_lat), method='cubic')

surface_fig = go.Figure(go.Surface(x=selected_grid_lon, y=selected_grid_lat, z=selected_grid_elev, colorscale='spectral'))
surface_fig.update_layout(title='Selected Area Bathymetric Surface Map (Plotly)', scene=dict(xaxis_title='Longitude', yaxis_title='Latitude', zaxis_title='Depth'))
surface_fig.update_traces(colorbar=dict(title='Depth', x=1.15))
surface_fig.show()


In [10]:
# DEFINING A THRESHOLD STATE
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

def find_tl(l):
    f,k,s,t,d,pH,r=l
    f1=0.78*((s/35)**0.5)*(2.7183**(t/26))
    f2=42*(2.7183**(t/17))
    absorption_loss=((0.106*f1*f*f*(2.7183**((pH-8)/0.56)))/((f1*f1)+(f*f)))+(0.52*(1+t/43)*(s/35)*(f2*f*f*(2.7183**(-d/6)))/((f2*f2)+(f*f)))+(0.00049*f*f*(2.7183**(-(t/27+d/17))))
    transmission_loss=10*k*np.log10(r*1000)+absorption_loss*r
    return absorption_loss,transmission_loss

data=[14,2,35,4,-df0['Depth'].min()/1000,8,0.25]# THRESHOLD STATE [frequency, spreading factor ,salinity, temperature, depth, pH, propagation range]

alpha,tl_th=find_tl(data)
print("Threshold Transmission Loss:",tl_th)
print("Absorption Loss:",alpha)
# TESTING A SAMPLE STATE
def equation(r_new, tl_th):
    return 20 * np.log10(r_new) + 0.001*alpha * r_new - tl_th

def find_r_new(tl, initial_guess=1.0):
    r_new_solution = fsolve(equation, initial_guess, args=(tl,))
    return r_new_solution[0]
#print(find_r_new(34.081501776982066))

new_data=[14,2,35,4,0.215,8,0.25]
new_losses=find_tl(new_data)
print(f"New transmission loss: {new_losses[1]}")

if new_losses[1]>tl_th:
    loss=2*tl_th-new_losses[1]
else:
    loss=tl_th
print("New Propagation Range(in m):",find_r_new(loss))

Threshold Transmission Loss: 48.293928102648515
Absorption Loss: 1.3405117168310636
New transmission loss: 48.47606157226142
New Propagation Range(in m): 245.001293047154


In [11]:
df=pd.read_csv('filtered_area.csv')
print(df)
df['Losses']=find_tl([14,2,35,4,-df['Depth']/1000,8,0.25])[1]
df['Range'] = df.apply(lambda row: find_r_new(2*tl_th-row['Losses'], alpha) if row['Losses'] > tl_th else 250, axis=1)
df = df.sort_values(by=['Latitude', 'Longitude']).reset_index(drop=True)
df

     Latitude  Longitude  Depth
0   13.785536  86.772199  -3065
1   13.786619  86.769529  -3065
2   13.786328  86.748916  -3066
3   13.766136  86.747290  -3067
4   13.764261  86.749535  -3067
5   13.761619  86.754013  -3067
6   13.761191  86.758254  -3067
7   13.760049  86.771480  -3067
8   13.782401  86.771861  -3065
9   13.783813  86.769018  -3065
10  13.784924  86.765351  -3065
11  13.785844  86.759668  -3065
12  13.784836  86.755903  -3065
13  13.783996  86.752147  -3066
14  13.779943  86.747867  -3066
15  13.770597  86.748880  -3067
16  13.768580  86.751733  -3067
17  13.766586  86.755424  -3067
18  13.764802  86.760671  -3067
19  13.763905  86.765920  -3067
20  13.763798  86.768424  -3067
21  13.764078  86.770954  -3067
22  13.779648  86.772776  -3066
23  13.781234  86.768034  -3065
24  13.781941  86.763173  -3065
25  13.782649  86.758310  -3065
26  13.781169  86.754082  -3066
27  13.778768  86.751263  -3066
28  13.775423  86.750561  -3067
29  13.773543  86.751141  -3067
30  13.7

,Latitude,Longitude,Depth,Losses,Range
0,13.760049,86.771480,-3067,48.294028,249.997229
1,13.761191,86.758254,-3067,48.294028,249.997229
2,13.761619,86.754013,-3067,48.294028,249.997229
3,13.763798,86.768424,-3067,48.294028,249.997229
4,13.763905,86.765920,-3067,48.294028,249.997229
5,13.764078,86.770954,-3067,48.294028,249.997229
6,13.764261,86.749535,-3067,48.294028,249.997229
7,13.764802,86.760671,-3067,48.294028,249.997229
8,13.766136,86.747290,-3067,48.294028,249.997229
9,13.766586,86.755424,-3067,48.294028,249.997229


In [12]:
selected_sensor = pd.DataFrame(columns=['Latitude', 'Longitude', 'Depth','Total Depth','Range'])

# Iterate over your data and add rows to the DataFrame
for i, j, depth, coverage_range in zip(df['Latitude'], df['Longitude'], df['Depth'], df['Range']):
    coverage_length = depth
    while coverage_length+coverage_range<0:
        coverage_length += coverage_range
        new_row = pd.DataFrame({'Latitude': [i], 'Longitude': [j], 'Depth': [coverage_length], 'Total Depth': depth,'Range': [coverage_range]})
        selected_sensor = pd.concat([selected_sensor, new_row], ignore_index=True)
selected_sensor = selected_sensor.sort_values(by=['Latitude', 'Longitude']).reset_index(drop=True)
selected_sensor

C:\Users\aksha\AppData\Local\Temp\ipykernel_8304\862810430.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,Latitude,Longitude,Depth,Total Depth,Range
0,13.760049,86.771480,-2817.002771,-3067,249.997229
1,13.760049,86.771480,-2567.005542,-3067,249.997229
2,13.760049,86.771480,-2317.008314,-3067,249.997229
3,13.760049,86.771480,-2067.011085,-3067,249.997229
4,13.760049,86.771480,-1817.013856,-3067,249.997229
...,...,...,...,...,...
631,13.786619,86.769529,-1065.044346,-3065,249.994457
632,13.786619,86.769529,-815.049889,-3065,249.994457
633,13.786619,86.769529,-565.055433,-3065,249.994457
634,13.786619,86.769529,-315.060976,-3065,249.994457


In [13]:
fig = go.Figure()

# Add surface map
fig.add_trace(go.Surface(
    x=df['Latitude'], y=df['Longitude'], z=df['Depth'],
    colorscale='turbo',
    opacity=1,
    showscale=False
))

# Add selected sensors as scatter plot
fig.add_trace(go.Scatter3d(
    x=selected_sensor['Latitude'], y=selected_sensor['Longitude'], z=selected_sensor['Depth'],
    mode='markers',
    marker=dict(
        size=4,
        color='red',
        symbol='circle'
    ),
    name='Selected Sensors'
))

# Customize layout
fig.update_layout(
    title='3D Surface Map with Selected Sensor Locations',
    scene=dict(
        xaxis_title='Latitude',
        yaxis_title='Longitude',
        zaxis_title='Depth',
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the plot
fig.show()

In [ ]:
#O(nlogn)
import pandas as pd
import numpy as np
import math
import heapq

# Function to calculate Euclidean distance considering latitude, longitude, and depth
def euclidean_distance_vectorized(lat1, lon1, depth1, lat2, lon2, depth2):
    x1 = lon1 * np.cos(np.radians(lat1))
    y1 = lat1
    z1 = depth1
    
    x2 = lon2 * np.cos(np.radians(lat2))
    y2 = lat2
    z2 = depth2

    horizontal_distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) * 111320  # Convert degrees to meters
    vertical_distance = z2 - z1

    distance = np.sqrt(horizontal_distance**2 + vertical_distance**2)
    
    return distance

# Precompute all distances between sensors and grids
def precompute_distances(sensor_df, grid_df):
    lat1, lon1, depth1 = sensor_df['Latitude'].values, sensor_df['Longitude'].values, sensor_df['Depth'].values
    lat2, lon2, depth2 = grid_df['Latitude'].values, grid_df['Longitude'].values, grid_df['Depth'].values
    
    dist_matrix = np.zeros((lat1.size, lat2.size))
    
    for i in range(lat1.size):
        dist_matrix[i, :] = euclidean_distance_vectorized(lat1[i], lon1[i], depth1[i], lat2, lon2, depth2)
        
    return dist_matrix

def calculate_coverage(sensor_indices, dist_matrix, range_matrix):
    covered = np.any(dist_matrix[sensor_indices, :] <= range_matrix[sensor_indices, :], axis=0)
    return np.sum(covered) / dist_matrix.shape[1]

# Initialize variables
latitude = selected_sensor['Latitude'].values
longitude = selected_sensor['Longitude'].values
depth = selected_sensor['Depth'].values
range_values = selected_sensor['Range'].values

# Precompute distances and range matrix
dist_matrix = precompute_distances(selected_sensor, selected_sensor)
range_matrix = np.minimum(range_values[:, None], range_values[None, :])

# Start placing sensors
sensors = []
current_coverage = 0
remaining_indices = np.arange(len(selected_sensor))

# Create a priority queue to efficiently find the sensor with maximum coverage increase
pq = []

while current_coverage < 1:
    max_coverage_increase = 0
    best_sensor_index = None
    
    # Evaluate potential sensors and push them into the priority queue
    for i in remaining_indices:
        candidate_sensors = sensors + [i]
        new_coverage = calculate_coverage(candidate_sensors, dist_matrix, range_matrix)
        
        coverage_increase = new_coverage - current_coverage
        if coverage_increase > max_coverage_increase:
            max_coverage_increase = coverage_increase
            best_sensor_index = i
    
    if best_sensor_index is None:
        break
    
    sensors.append(best_sensor_index)
    current_coverage += max_coverage_increase
    print("Coverage:", current_coverage)
    
    if current_coverage >= 1:
        break
    
    # Remove the covered points from remaining_indices
    covered_by_best_sensor = dist_matrix[best_sensor_index, :] <= range_values[best_sensor_index]
    remaining_indices = remaining_indices[~covered_by_best_sensor[remaining_indices]]

# Convert sensor list to dataframe
sensors_df = selected_sensor.iloc[sensors].reset_index(drop=True)

# Output the new dataframe with sensor placements
print(sensors_df)


Coverage: 0.006289308176100629
Coverage: 0.012578616352201259
Coverage: 0.018867924528301886
Coverage: 0.025157232704402517
Coverage: 0.031446540880503145
Coverage: 0.03773584905660377
Coverage: 0.0440251572327044
Coverage: 0.04874213836477988
Coverage: 0.05345911949685535
Coverage: 0.05817610062893082
Coverage: 0.06289308176100629
Coverage: 0.06761006289308176
Coverage: 0.07232704402515723
Coverage: 0.0770440251572327
Coverage: 0.08176100628930817
Coverage: 0.08647798742138364
Coverage: 0.09119496855345911
Coverage: 0.0959119496855346
Coverage: 0.10062893081761007
Coverage: 0.10534591194968554
Coverage: 0.11006289308176101
Coverage: 0.11477987421383648
Coverage: 0.11949685534591195
Coverage: 0.12421383647798742
Coverage: 0.1289308176100629
Coverage: 0.13364779874213836
Coverage: 0.13836477987421383
Coverage: 0.1430817610062893
Coverage: 0.14779874213836477
Coverage: 0.15251572327044025
Coverage: 0.15723270440251572
Coverage: 0.1619496855345912
Coverage: 0.16666666666666666
Coverage: 0

In [8]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter3d(
    x=sensors_df['Latitude'], y=sensors_df['Longitude'], z=sensors_df['Depth'],
    mode='markers',
    marker=dict(
        size=4,
        color='blue',  # You can choose any color you prefer
        symbol='circle'
    ),
    name='New Point'
))
fig3.update_layout(
    title='3D Surface Map with Selected Sensor Locations',
    scene=dict(
        xaxis_title='Latitude',
        yaxis_title='Longitude',
        zaxis_title='Depth',
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)